In [34]:
from my_trader import *
import math

In [2]:
robinhood = get_robinhood()
robinhood.login("lgyhz123","5093945464lgyhz")

In [3]:
beta(["AAPL"],interval = "minute")

(          Beta
 AAPL  0.746571, array([[  7.39559756e-07,   8.40389182e-07],
        [  8.40389182e-07,   1.71334738e-06]]))

In [3]:
robinhood.get_my_position_beta_minute()

ZNGA 1.0000
TAHO 1.0000
RRGB 1.0000
SPY 6.0000
IEP 3.0000
OSG 1.0000
ROYT 2.0000
FCFS 1.0000
AES 1.0000
ISSC 1.0000
FDC 1.0000
WLK 1.0000
VRNT 1.0000
0.840984907799
          Beta  Quantity  Last_price    Weight
ZNGA  0.036493       1.0        3.80  0.001752
TAHO -0.369054       1.0        4.72  0.002176
RRGB  0.403548       1.0       53.30  0.024568
SPY   1.000000       6.0      279.61  0.773311
IEP   0.325061       3.0       55.90  0.077301
OSG   0.210234       1.0        2.81  0.001295
ROYT  0.105414       2.0        2.52  0.002323
FCFS  0.402574       1.0       70.95  0.032704
AES   0.222917       1.0       10.72  0.004941
ISSC  0.054599       1.0        3.03  0.001397
FDC   0.435015       1.0       17.60  0.008113
WLK   0.137103       1.0      108.77  0.050137
VRNT  0.405705       1.0       43.35  0.019982


(0.84098490779876256,           Beta  Quantity  Last_price    Weight
 ZNGA  0.036493       1.0        3.80  0.001752
 TAHO -0.369054       1.0        4.72  0.002176
 RRGB  0.403548       1.0       53.30  0.024568
 SPY   1.000000       6.0      279.61  0.773311
 IEP   0.325061       3.0       55.90  0.077301
 OSG   0.210234       1.0        2.81  0.001295
 ROYT  0.105414       2.0        2.52  0.002323
 FCFS  0.402574       1.0       70.95  0.032704
 AES   0.222917       1.0       10.72  0.004941
 ISSC  0.054599       1.0        3.03  0.001397
 FDC   0.435015       1.0       17.60  0.008113
 WLK   0.137103       1.0      108.77  0.050137
 VRNT  0.405705       1.0       43.35  0.019982)

In [15]:
beta(["UVXY"],bench= "^VIX",interval = "day")

(          Beta
 UVXY  0.846917, array([[ 0.00338583,  0.00206093],
        [ 0.00206093,  0.00174895]]))

In [9]:
robinhood.logout()

In [5]:
bench= "SPY"
ticker_list = ["UVXY"]
interval = "minute"

In [6]:
robinhood = get_robinhood()
betas = []
ben_mark=pd.DataFrame()
if interval == "day":
    ben_mark= da.DataReader(bench,"yahoo",datetime.now()-timedelta(days=90),datetime.now())
    ben_mark=ben_mark.rename(columns ={"Adj Close":bench})
elif interval == "minute":
    ben_mark = robinhood.get_historical(bench,interval="10minute",span = "week")
    ben_mark=ben_mark.rename(columns ={"close_price":bench})
    ben_mark[bench]=ben_mark[bench].astype(float)
    ben_mark.index = ben_mark.begins_at
ben_mark[bench + "_re"] = log(ben_mark[bench]/ben_mark[bench].shift(1))
for i in list(ticker_list):
    new=[]
    ticker = i
    try:
        if interval =="day":

            stock = da.DataReader(str(ticker),"yahoo",datetime.now()-timedelta(days=90),datetime.now())
            stock = stock.rename(columns = {"Adj Close":ticker})
        elif interval == "minute":
            stock= robinhood.get_historical(ticker,interval="10minute",span = "week")
            stock=stock.rename(columns ={"close_price":ticker})
            stock[ticker] = stock[ticker].astype(float)
            stock.index = stock.begins_at
    except:
        print (str(i)+" ticker maybe wrong. Error in getting data")
        betas.append(np.NaN)
        continue

    # get return and put them in a new dataframe

    stock[ticker + "_re"] = log(stock[ticker]/stock[ticker].shift(1))
    new = pd.concat([ben_mark,stock],axis =1)
    new = new[[bench + "_re",ticker + "_re" ]]
    new = new.dropna()

    #calculate beta using covariance matrix
    covmat = np.cov(new[bench + "_re"],new[ticker + "_re"])
    beta = covmat[0,1]/  np.sqrt(covmat[1,1]*covmat[0,0])
    betas.append(beta)
betas = pd.DataFrame(betas)
betas.index = ticker_list
betas.columns=["Beta"]


In [22]:
robinhood.my_trader.market_value()

2169.45

In [6]:
robinhood.my_trader.quotes_data(["AAPL","UVXY"])

[{u'adjusted_previous_close': u'176.1900',
  u'ask_price': u'179.4600',
  u'ask_size': 100,
  u'bid_price': u'179.3000',
  u'bid_size': 200,
  u'has_traded': True,
  u'instrument': u'https://api.robinhood.com/instruments/450dfc6d-5510-4d40-abfb-f633b7d9be3e/',
  u'last_extended_hours_trade_price': u'179.3700',
  u'last_trade_price': u'179.1000',
  u'last_trade_price_source': u'consolidated',
  u'previous_close': u'176.1900',
  u'previous_close_date': u'2018-01-16',
  u'symbol': u'AAPL',
  u'trading_halted': False,
  u'updated_at': u'2018-01-18T01:00:00Z'},
 {u'adjusted_previous_close': u'9.7500',
  u'ask_price': u'9.4600',
  u'ask_size': 1600,
  u'bid_price': u'9.4200',
  u'bid_size': 500,
  u'has_traded': True,
  u'instrument': u'https://api.robinhood.com/instruments/00e90099-4281-4c93-b50d-fbd4d2469821/',
  u'last_extended_hours_trade_price': u'9.4000',
  u'last_trade_price': u'9.3400',
  u'last_trade_price_source': u'consolidated',
  u'previous_close': u'9.7500',
  u'previous_close_

In [23]:
data = pd.read_csv("file/Trade_suggestion2018-01-17.csv")

In [42]:
data["position_place"]= np.ceil(((data.Close.sum()/len(data.Close))/data.Close))

In [44]:
np.dot(data.position_place,data.Close)

688.95499500000005